# Step 1. 피치 정확도 구하기

In [264]:
import mido

# Step 1. 피치 정확도 구하기

def calculate_pitch_accuracy(input_midi_path, target_midi_path, limit_offset=0):
    """
    인풋 MIDI와 정답 MIDI의 pitch 정확도 계산 - Yeong-Min Ko
    
    Args:
        input_midi: 인풋 MIDI 파일
        target_midi: 타겟 MIDI 파일
        
    Return:
        pitch 정확도
    """

    # MIDI 파일 읽기
    input_midi = mido.MidiFile(input_midi_path)
    target_midi = mido.MidiFile(target_midi_path)

    # 인풋 MIDI와 타겟 MIDI의 음표 정보를 추출
    input_notes = [msg.note for msg in input_midi if msg.type == 'note_on']
    target_notes = [msg.note for msg in target_midi if msg.type == 'note_on']
    
    # 피치 정확도 계산
    total_accuracy = 0
    for input_note in input_notes:
        for reference_note in target_notes:
            if abs(input_note - reference_note) < limit_offset:
                total_accuracy += 1
                break

    # 전체 음표 수와 정확한 음표 수를 이용하여 정확도 계산
    pitch_accuracy = (total_accuracy / len(target_notes)) * 100
    if pitch_accuracy > 100:
        pitch_accuracy %= 100
    
    return pitch_accuracy

def extract_note_info(midi_file_path):
    """
    MIDI 파일에서 노트 정보를 추출
    """
    mid = mido.MidiFile(midi_file_path)

    note_info = []  # 각 노트의 정보를 저장할 리스트

    current_time = 0  # 현재 시간 초기화

    for i, track in enumerate(mid.tracks):
        for msg in track:
            current_time += msg.time  # 현재 시간을 누적

            if msg.type == 'note_on':
                note_start = current_time  # 노트 시작 시간
                note_info.append({'start': note_start, 'end': None, 'note': msg.note})
            elif msg.type == 'note_off':
                note_end = current_time  # 노트 종료 시간
                # 현재 노트 정보에 종료 시간 업데이트
                note_info[-1]['end'] = note_end

    return note_info

def display_note_info(note_info, title):
    """
    노트 정보를 출력
    """
    print(f'\nNote information for {title}:')
    print('-----------------------------------')
    for idx, note in enumerate(note_info):
        print(f'Note {idx + 1}:')
        print(f'Start Time: {note["start"]} ms')
        print(f'End Time: {note["end"]} ms')
        print(f'Note Pitch: {note["note"]}')
        print('-----------------------------------')

def calculate_accuracy_level(accuracy):
    """
    피치 정확도에 따라 Level을 계산하는 함수

    Args:
        accuracy: 피치 정확도

    Return:
        계산된 Level
    """
    level = int(min(10, round(accuracy / 10)))  # 10% 단위로 Level 계산, 최대 Level은 10, 반올림 적용
    return level

# 메인 시작 부분
if __name__ == "__main__":
    # Step 2_1. Mido 라이브러리로 정답 데이터와 같은 인풋 데이터 읽기
    input_midi_path1 = 'Prelude1.mid'
    target_midi_path1 = 'Prelude1.mid'
    limit_offset = 5 # 정확도를 판단할 때 허용되는 피치 차이
    accuracy1 = calculate_pitch_accuracy(input_midi_path1, target_midi_path1, limit_offset)

    # Step 2_2. Mido 라이브러리로 정답 데이터와 다른 인풋 데이터 읽기
    input_midi_path2 = 'Prelude1.mid'
    target_midi_path2 = 'Fugue1.mid'
    limit_offset = 1
    accuracy2 = calculate_pitch_accuracy(input_midi_path2, target_midi_path2, limit_offset)

    # Step 2_3. Mido 라이브러리로 정답 데이터와 다른 인풋 데이터 읽기(피치 한계 약간 줬을 때)
    input_midi_path3 = 'Prelude1.mid'
    target_midi_path3 = 'Fugue1.mid'
    limit_offset = 10
    accuracy3 = calculate_pitch_accuracy(input_midi_path3, target_midi_path3, limit_offset)

    # Step 2_4. Mido 라이브러리로 정답 데이터와 다른 인풋 데이터 읽기(피치 한계 많이 줬을 때)
    input_midi_path4 = 'Fugue3.mid'
    target_midi_path4 = 'Fugue1.mid'
    limit_offset = 1000
    accuracy4 = calculate_pitch_accuracy(input_midi_path4, target_midi_path1, limit_offset)

    # Step 3. 각각의 정확도 및 Level 출력
    level1 = calculate_accuracy_level(accuracy1)
    level2 = calculate_accuracy_level(accuracy2)
    level3 = calculate_accuracy_level(accuracy3)
    level4 = calculate_accuracy_level(accuracy4)
    
    print(f'Pitch Accuracy testCase 1: {accuracy1:.2f}% - Level {level1}')
    print(f'Pitch Accuracy testCase 2: {accuracy2:.2f}% - Level {level2}')
    print(f'Pitch Accuracy testCase 3: {accuracy3:.2f}% - Level {level3}')
    print(f'Pitch Accuracy testCase 4: {accuracy4:.2f}% - Level {level4}')
    
    # 각 MIDI 파일의 노트 정보 추출
    note_info_1 = extract_note_info(midi_file_path_1)
    note_info_2 = extract_note_info(midi_file_path_2)
    note_info_4 = extract_note_info(midi_file_path_4)

    # 노트 정보 출력
    display_note_info(note_info_1, 'Prelude1')
    display_note_info(note_info_2, 'Fugue1')
    display_note_info(note_info_4, 'Fugue3')

Pitch Accuracy testCase 1: 100.00% - Level 10
Pitch Accuracy testCase 2: 71.56% - Level 7
Pitch Accuracy testCase 3: 73.56% - Level 7
Pitch Accuracy testCase 4: 62.43% - Level 6

Note information for Prelude1:
-----------------------------------
Note 1:
Start Time: 32880 ms
End Time: 32940 ms
Note Pitch: 67
-----------------------------------
Note 2:
Start Time: 32940 ms
End Time: 33000 ms
Note Pitch: 72
-----------------------------------
Note 3:
Start Time: 33000 ms
End Time: 33060 ms
Note Pitch: 76
-----------------------------------
Note 4:
Start Time: 33060 ms
End Time: 33120 ms
Note Pitch: 67
-----------------------------------
Note 5:
Start Time: 33120 ms
End Time: 33180 ms
Note Pitch: 72
-----------------------------------
Note 6:
Start Time: 33180 ms
End Time: 33240 ms
Note Pitch: 76
-----------------------------------
Note 7:
Start Time: 33360 ms
End Time: 33420 ms
Note Pitch: 67
-----------------------------------
Note 8:
Start Time: 33420 ms
End Time: 33480 ms
Note Pitch: 7

# Step 2. 곡의 빠르기 유사도 점수 구하기

In [265]:
import mido
from mido import MidiFile

def extract_tempo(midi_file):
    """
    MIDI 파일에서 템포 정보를 추출
    """
    mid = MidiFile(midi_file)

    tempo_list = []  # 각 트랙의 템포를 저장할 리스트

    for i, track in enumerate(mid.tracks):
        for msg in track:
            if msg.type == 'set_tempo':
                # 템포 메시지에서 마이크로초당 박자 속도를 추출
                tempo = mido.tempo2bpm(msg.tempo)
                tempo_list.append(tempo)

    # 템포 정보가 없는 경우 기본값으로 None을 반환
    if not tempo_list:
        return None

    # 전체 트랙의 평균 템포를 계산
    average_tempo = sum(tempo_list) / len(tempo_list)

    return average_tempo

def calculate_tempo_score(tempo_difference, threshold = 3):
    """
    두 곡간의 평균 템포 차이를 기반으로 점수를 빠르기 Score를 계산하는 함수
    
    Arg:
        tempo_difference: 템포 차이

    Returns:
        계산된 Score
    """
    excess_difference = max(0, tempo_difference - threshold) # 임계값을 초과한 템포 차이

    if tempo_difference <= threshold:
        # threshold 이하의 템포 차이는 최고 점수 100을 부여
        score = 100
    else:
        # threshold를 초과하는 경우 템포 차이에 따라 점수 부여
        # 차후에 가중치 생각해 볼 필요 있음 - 현재는 임시 로직
        score = max(0, 100 - min(3, excess_difference) * (tempo_difference - threshold))

    return f'{score:.3f}'

def calculate_tempo_level(score):
    """
    두 곡 간의 평균 템포 차이를 기반으로 Level을 계산하는 함수

    Arg:
        score: 현재 점수

    Returns:
        계산된 Level
    """
    score = float(score)  # 문자열에서 숫자로 변환
    level = int(min(10, round(score / 10))) # 최대 Level은 10
    return level

if __name__ == "__main__":
    # 두 MIDI 파일의 경로를 지정
    midi_file_path_1 = 'Prelude1.mid'
    midi_file_path_2 = 'Fugue1.mid'
    midi_file_path_3 = 'Fugue1.mid'
    midi_file_path_4 = 'Fugue3.mid'

    # 두 파일의 평균 템포를 추출
    average_tempo_1 = extract_tempo(midi_file_path_1)
    average_tempo_2 = extract_tempo(midi_file_path_2)
    average_tempo_3 = extract_tempo(midi_file_path_3)    
    average_tempo_4 = extract_tempo(midi_file_path_4)

    if average_tempo_1 is not None and average_tempo_2 is not None and average_tempo_3 is not None and average_tempo_3 is not None:
        # 평균 템포의 차이를 계산
        tempo_difference1 = abs(average_tempo_1 - average_tempo_2) # 다른 파일의 템포 평균
        tempo_difference2 = abs(average_tempo_3 - average_tempo_2) # 같은 파일의 템포 평균
        tempo_difference3 = abs(average_tempo_4 - average_tempo_3) # 다른 파일의 템포 평균
        
        # 템포 차이를 기반으로 점수 계산
        score1 = calculate_tempo_score(tempo_difference1) # 다른 파일로 테스트
        score2 = calculate_tempo_score(tempo_difference2) # 같은 파일로 테스트
        score3 = calculate_tempo_score(tempo_difference3) # 다른 파일로 테스트
        
        # 템포 차이에 따른 Level 계산 및 출력
        level_tempo1 = calculate_tempo_level(score1)
        level_tempo2 = calculate_tempo_level(score2)
        level_tempo3 = calculate_tempo_level(score3)
        
        print('Speed similarity scores of songs')
        print('--------------------------------------------------------------------------------------------')
        print(f'Average Tempo of {midi_file_path_1}: {average_tempo_1} BPM')
        print(f'Average Tempo of {midi_file_path_2}: {average_tempo_2} BPM')
        print(f'Average Tempo of {midi_file_path_4}: {average_tempo_3} BPM')
        print('--------------------------------------------------------------------------------------------')
        print(f'Tempo Difference Between {midi_file_path_1} and {midi_file_path_2}: {tempo_difference1} BPM')
        print(f'Tempo Difference Level testCase 1: Score: {score1} - Level: {level_tempo1}') # 다른 파일로 연주한 경우
        print('--------------------------------------------------------------------------------------------')
        print(f'Tempo Difference Between {midi_file_path_2} and {midi_file_path_3}: {tempo_difference2} BPM')
        print(f'Tempo Difference Level testCase 2: Score: {score2} - Level: {level_tempo2}') # 같은 파일로 연주한 경우
        print('--------------------------------------------------------------------------------------------')   
        print(f'Tempo Difference Between {midi_file_path_3} and {midi_file_path_4}: {tempo_difference3} BPM')
        print(f'Tempo Difference Level testCase 1: Score: {score3} - Level: {level_tempo3}') # 다른 파일로 연주한 경우
        print('--------------------------------------------------------------------------------------------')
    else:
        print('하나 또는 두 파일 모두의 평균 템포를 계산할 수 없습니다.')

Speed similarity scores of songs
--------------------------------------------------------------------------------------------
Average Tempo of Prelude1.mid: 63.30000225000275 BPM
Average Tempo of Fugue1.mid: 53.12499922083624 BPM
Average Tempo of Fugue3.mid: 53.12499922083624 BPM
--------------------------------------------------------------------------------------------
Tempo Difference Between Prelude1.mid and Fugue1.mid: 10.175003029166511 BPM
Tempo Difference Level testCase 1: Score: 78.475 - Level: 8
--------------------------------------------------------------------------------------------
Tempo Difference Between Fugue1.mid and Fugue1.mid: 0.0 BPM
Tempo Difference Level testCase 2: Score: 100.000 - Level: 10
--------------------------------------------------------------------------------------------
Tempo Difference Between Fugue1.mid and Fugue3.mid: 18.43055086620948 BPM
Tempo Difference Level testCase 1: Score: 53.708 - Level: 5
-----------------------------------------------

In [266]:
# 아래 코드는 바로 위 로직짜기 전에 작성한 연습 코드
import mido
from mido import MidiFile

def extract_tempo(midi_file):
    """
    MIDI 파일에서 템포 정보를 추출
    """
    mid = MidiFile(midi_file)

    for i, track in enumerate(mid.tracks):
        for msg in track:
            if msg.type == 'set_tempo':
                # 템포 메시지에서 마이크로초당 박자 속도를 추출
                tempo = mido.tempo2bpm(msg.tempo)
                return tempo

    # 템포 정보가 없는 경우 기본값으로 120 BPM을 반환.
    return 120

def tempo_difference(tempo1, tempo2):
    """
    두 템포 값 간의 유사도를 계산
    """
    # 예제로 간단하게 차이의 절대값을 반환
    return abs(tempo1 - tempo2)

if __name__ == "__main__":
    # 두 MIDI 파일의 경로를 지정.
    midi_file_path_1 = 'Prelude1.mid'
    midi_file_path_2 = 'Fugue1.mid'

    # 템포 정보를 추출
    tempo1 = extract_tempo(midi_file_path_1)
    tempo2 = extract_tempo(midi_file_path_2)

    # 템포 유사도를 계산
    similarity = tempo_difference(tempo1, tempo2)

    print(f'Tempo of {midi_file_path_1}: {tempo1} BPM')
    print(f'Tempo of {midi_file_path_2}: {tempo2} BPM')
    print(f'Tempo Difference Between {midi_file_path_1} And {midi_file_path_2}: {difference} BPM')

Tempo of Prelude1.mid: 72.00002880001152 BPM
Tempo of Fugue1.mid: 64.0 BPM
Tempo Difference Between Prelude1.mid And Fugue1.mid: 8.00002880001152 BPM


# Step 3. 노트의 길이 유사도 점수 구하기
- 조금 더 고민해봐야 할 것 같음

In [342]:
# 유사도 ?

def extract_note_durations(midi_file):
    """
    MIDI 파일에서 각 노트의 길이를 추출
    """
    mid = MidiFile(midi_file)

    note_durations = []  # 각 노트의 길이를 저장할 리스트

    current_time = 0  # 현재 시간 초기화

    for i, track in enumerate(mid.tracks):
        for msg in track:
            current_time += msg.time  # 현재 시간을 누적

            if msg.type == 'note_on':
                note_start = current_time  # 노트 시작 시간
            elif msg.type == 'note_off':
                note_end = current_time  # 노트 종료 시간
                note_duration = note_end - note_start  # 노트 길이 계산
                note_durations.append(note_duration)

    return note_durations

def adjust_note_lengths(note_durations_1, note_durations_2):
    """
    두 노트 길이 리스트의 길이를 맞추기
    """
    min_length = min(len(note_durations_1), len(note_durations_2))
    return note_durations_1[:min_length], note_durations_2[:min_length]

def calculate_note_duration_similarity(note_durations_1, note_durations_2, threshold = 10):
    """
    두 곡 간의 노트 길이 유사도 점수를 계산
    """
    # 노트 길이 리스트의 길이를 맞춤
    note_durations_1, note_durations_2 = adjust_note_lengths(note_durations_1, note_durations_2)

    # 노트 길이 차이의 절대값을 계산하여 유사도 점수 산출
    absolute_differences = [abs(a - b) for a, b in zip(note_durations_1, note_durations_2)]
    average_difference = sum(absolute_differences) / len(absolute_differences)
    if average_difference < threshold:
        return 100, note_durations_1, note_durations_2
    else:
        return average_difference % 100, note_durations_1, note_durations_2

if __name__ == "__main__":
    # 두 MIDI 파일의 경로를 지정
    midi_file_path_1 = 'Fugue1.mid'
    midi_file_path_2 = 'Fugue3.mid'
    midi_file_path_3 = 'Prelude1.mid'
    

    # 각 MIDI 파일의 노트 길이를 추출
    note_durations_1 = extract_note_durations(midi_file_path_1)
    note_durations_2 = extract_note_durations(midi_file_path_2)
    note_durations_3 = extract_note_durations(midi_file_path_3)

    # 노트 길이 유사도 점수 계산 및 노트 길이 리스트 출력
    similarity_score, note_durations_1, note_durations_2 = calculate_note_duration_similarity(note_durations_1, note_durations_2)
    similarity_score2, note_durations_2, note_durations_3 = calculate_note_duration_similarity(note_durations_2, note_durations_3)
    similarity_score3, note_durations_3, note_durations_3 = calculate_note_duration_similarity(note_durations_3, note_durations_3)

    if similarity_score is not None and similarity_score2 is not None and similarity_score3 is not None:
        print(f'Note Duration Similarity Score: {similarity_score:.2f}%')
        print(f'Note Duration Similarity Score 2: {similarity_score2:.2f}%')
        print(f'Note Duration Similarity Score 3: {similarity_score3:.2f}%')
        print('-----------------------------------')
        print('Note Durations 1\n', note_durations_1)
        print('-----------------------------------')
        print('Note Durations 2\n', note_durations_2)
        print('-----------------------------------')
        print('Note Durations 3\n', note_durations_3)
    else:
        print('Error')


Note Duration Similarity Score: 71.50%
Note Duration Similarity Score 2: 8.58%
Note Duration Similarity Score 3: 100.00%
-----------------------------------
Note Durations 1
 [120, 120, 120, 180, 30, 2, 120, 120, 120, 170, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 120, 120, 120, 60, 60, 60, 60, 170, 60, 60, 60, 60, 60, 60, 60, 60, 60, 480, 240, 120, 120, 120, 180, 30, 30, 120, 120, 120, 180, 60, 60, 60, 120, 180, 60, 60, 60, 480, 180, 60, 480, 240, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 120, 120, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 60, 60, 120, 120, 120, 240, 60, 60, 15, 15, 15, 15, 15, 15, 15, 15, 15, 45, 50, 240, 120, 120, 120, 180, 30, 2, 120, 120, 120, 120, 180, 30, 2, 120, 120, 120, 180, 60, 60, 60, 120, 120, 120, 120, 120, 60, 60, 60, 60, 60, 60, 60, 55, 5, 1, 30, 30, 2, 60, 60, 60, 60, 3, 1, 1, 1, 1, 100, 50, 148, 120, 120, 120, 147, 30, 2, 120, 120, 120, 170, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 3, 1, 